In [5]:
## 模型与剖分

import math
import numpy as np
from numpy.linalg import solve
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

from tool import PDE, getIsBdNode, uniform_refine, get_cr_node_cell
from tool import get_A1_A2_F, my_solve, error, print_error, print_P
from tool import get_stiff_and_div_matrix, get_bb

n   = 4  #剖分次数
n   = n + 1 
Lam = [1,1e1,1e2,1e3,1e4,1e5]
Mu  = Lam

H     = np.zeros(n)                              #步长
P     = np.zeros(len(Lam))                       #误差阶
E     = np.zeros((len(Lam),n), dtype=np.float64) #每个lambda(行)对应的误差(列)
    
for i in range(len(Lam)):
    pde = PDE(Mu[i], Lam[i])# / 2 + 10, Lam[i])
    node = np.array([
            (0,0),
            (1,0),
            (1,1),
            (0,1)], dtype=np.float64)
    cell = np.array([(1,2,0), (3,0,2)], dtype=np.int64)
    for j in range(n):
        NC = cell.shape[0]
        #print("cr_NC= ", NC)
        # nn 特定情况下剖分次数
        nn  = math.log(NC/2, 4)
        NN = int(3 * 2 * 4**nn - (3 * 2 * 4**nn - 4 * 2**nn) / 2)
        cm = np.ones(NC, dtype=np.float64) / NC
     
        cr_node, cr_cell = get_cr_node_cell(node,cell)
        #print("cr_node= ", cr_node.shape[0])
        #print("cr_cell= ", cr_cell.shape[0])

        # 单元刚度矩阵和单元载荷向量 
        A1, A2 = get_stiff_and_div_matrix(cr_node, cr_cell, cm)
        bb = get_bb(pde, node, cell, cm)
        
        A1, A2, F = get_A1_A2_F(A1, A2, bb, cr_node, cr_cell)
        A = pde.mu * A1 + (pde.lam + pde.mu) * A2
              
        uh = my_solve(A, F, cr_node, getIsBdNode)
        u = pde.solution(cr_node)
        H[j] = np.sqrt(2 * cm[0])
        #print("h= ", np.sqrt(2 * cm[0]))
        # 计算误差
        E[i][j] = error(u, uh)
        #E[i][j] = H_1_norm(u-uh, H[j])
        #print("u= ", u)
        #print("uh= ", uh)
        node, cell = uniform_refine(node, cell)

if n-1 > 1: 
    for i in range(len(Lam)):
        fig = plt.figure()
        plt.plot(np.log(H[1:]), np.log(E[i][1:]))
        plt.title("lam={}".format(Lam[i]))
        plt.xlabel("log(h)")
        plt.ylabel("log(e)")
        plt.savefig(fname="CRFem/elasticityCRFemLam_{}.png"
                    .format(Lam[i]))
        plt.close(fig)
        
    for i in range(len(Lam)):
        f = np.polyfit(np.log(H[1:]), np.log(E[i][1:]) ,1)
        P[i] = f[0]

#print("u= ", np.max(u))
#print("uh= ", np.max(uh))
print_error(Lam, H, E)
#print_error(Lam, H, E[isInterNode], E_rel[isInterNode])
if n-1 > 1:
    print_P(Lam, P)

---------------------Lam= 1---------------------

h=  1.0
e= 0.29011206542651247

h=  0.5
e= 0.07987830915813865

h=  0.25
e= 0.020326196583183402

h=  0.125
e= 0.0058054494674396225

h=  0.0625
e= 0.0015808927711673066


---------------------Lam= 10.0---------------------

h=  1.0
e= 0.2901120654265124

h=  0.5
e= 0.07987830915813865

h=  0.25
e= 0.02032619658318347

h=  0.125
e= 0.0058054494674396225

h=  0.0625
e= 0.001580892771167297


---------------------Lam= 100.0---------------------

h=  1.0
e= 0.29011206542651247

h=  0.5
e= 0.07987830915813868

h=  0.25
e= 0.020326196583183458

h=  0.125
e= 0.0058054494674396155

h=  0.0625
e= 0.0015808927711673006


---------------------Lam= 1000.0---------------------

h=  1.0
e= 0.29011206542651247

h=  0.5
e= 0.07987830915813868

h=  0.25
e= 0.020326196583183444

h=  0.125
e= 0.005805449467439609

h=  0.0625
e= 0.0015808927711673058


---------------------Lam= 10000.0---------------------

h=  1.0
e= 0.29011206542651247

h=  0.5
e= 0.079